In [49]:
import pandas as pd
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt
from google.colab                import drive

pd.set_option('display.max_columns', 500)

In [50]:
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [51]:
df_am = pd.read_csv('/content/gdrive/MyDrive/Datasets/base_covid_AM/base.csv', sep=';', low_memory=False, parse_dates=['_dt_notificacao','_dt_evolucao','_dt_sintomas'])
df_am = df_am.drop(['_etnia', '_faixa etária', '_raca', '_sexo', '_srag_agente', '_srag', '_taxa', '_teste_anticorpo',
                    '_teste_antigeno', '_origem','_conclusao', '_se_notificacao','_criterio','_teste_pcr', '_evolução',
                    '_distrito', '_bairro_mapa','_dt_evolucao', '_dt_notificacao', '_dt_sintomas'], axis = 1)

## Tratamento de dados

In [54]:
df_am[['_comorb_cardio', '_comorb_cromossomica',
       '_comorb_diabetes', '_comorb_hemato', '_comorb_hepatica',
       '_comorb_imuno', '_comorb_neurologica', '_comorb_obessidade',
       '_comorb_renal', '_comorb_respiratoria','_gestante', '_profiss_saude',
       '_sintoma_dispneia', '_sintoma_febre', '_sintoma_garganta',
       '_sintoma_tosse','_sintoma_olfato', '_sintoma_paladar','_sintoma_outros']] = df_am[['_comorb_cardio', '_comorb_cromossomica',
       '_comorb_diabetes', '_comorb_hemato', '_comorb_hepatica',
       '_comorb_imuno', '_comorb_neurologica', '_comorb_obessidade',
       '_comorb_renal', '_comorb_respiratoria','_gestante', '_profiss_saude',
       '_sintoma_dispneia', '_sintoma_febre', '_sintoma_garganta',
       '_sintoma_tosse','_sintoma_olfato', '_sintoma_paladar', '_sintoma_outros']].fillna('Não')

In [55]:
df_am = df_am[(df_am['_idade'] >= 18.0) & (df_am['_idade'] <= 100.0)]
df_am['_idade'] = df_am['_idade'].fillna(df_am['_idade'].mean())
df_am = df_am.dropna()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


### Feature Engineering

In [64]:
df_am['grupos'] = df_am['_idade'] >= 60.0

In [66]:
df_am.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 556956 entries, 1 to 634837
Data columns (total 23 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   _classificacao        556956 non-null  object 
 1   _comorb_cardio        556956 non-null  object 
 2   _comorb_cromossomica  556956 non-null  object 
 3   _comorb_diabetes      556956 non-null  object 
 4   _comorb_hemato        556956 non-null  object 
 5   _comorb_hepatica      556956 non-null  object 
 6   _comorb_imuno         556956 non-null  object 
 7   _comorb_neurologica   556956 non-null  object 
 8   _comorb_obessidade    556956 non-null  object 
 9   _comorb_renal         556956 non-null  object 
 10  _comorb_respiratoria  556956 non-null  object 
 11  _idade                556956 non-null  float64
 12  _gestante             556956 non-null  object 
 13  _profiss_saude        556956 non-null  object 
 14  _sintoma_dispneia     556956 non-null  object 
 15  

In [65]:
df_am.head(12)

,_classificacao,_comorb_cardio,_comorb_cromossomica,_comorb_diabetes,_comorb_hemato,_comorb_hepatica,_comorb_imuno,_comorb_neurologica,_comorb_obessidade,_comorb_renal,_comorb_respiratoria,_idade,_gestante,_profiss_saude,_sintoma_dispneia,_sintoma_febre,_sintoma_garganta,_sintoma_tosse,_bairro,_sintoma_olfato,_sintoma_paladar,_sintoma_outros,grupos
1,Confirmado,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,26.0,Não,Não,Sim,Sim,Sim,Sim,TARUMA-ACU,Não,Não,Sim,False
2,Descartado,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,27.0,Não,Não,Sim,Sim,Sim,Sim,JORGE TEIXEIRA,Não,Não,Sim,False
3,Descartado,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,94.0,Não,Não,Sim,Sim,Sim,Sim,FLORES,Não,Não,Sim,True
8,Descartado,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,28.0,Não,Não,Sim,Sim,Não,Sim,CIDADE DE DEUS,Não,Não,Sim,False
9,Descartado,Sim,Não,Não,Não,Não,Não,Não,Não,Não,Não,80.0,Não,Não,Sim,Sim,Não,Sim,PETROPOLIS,Não,Não,Sim,True
10,Confirmado,Sim,Não,Sim,Não,Não,Não,Não,Não,Não,Sim,74.0,Não,Não,Sim,Sim,Sim,Sim,CACHOEIRINHA,Não,Não,Sim,True
11,Descartado,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,23.0,Sim,Não,Não,Sim,Sim,Sim,SANTA ETELVINA,Não,Não,Sim,False
12,Confirmado,Não,Não,Não,Sim,Não,Não,Não,Não,Não,Não,26.0,Não,Não,Sim,Sim,Sim,Sim,MONTE DAS OLIVEIRAS,Não,Não,Sim,False
14,Descartado,Não,Não,Sim,Não,Não,Não,Não,Não,Não,Não,85.0,Não,Não,Sim,Sim,Não,Sim,PONTA NEGRA,Não,Não,Sim,True
18,Descartado,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,49.0,Não,Não,Sim,Sim,Sim,Sim,TARUMA,Não,Não,Sim,False
